1. 문서 내용 읽기
2. 문서 분할(쪼개기)
    - 문서를 분할하지 않으면,
        - 토큰 수 초과로 답변을 생성하지 못할 수 있음
        - 문서 길이(input)가 길어서 답변 생성에 시간 소요됨 -> 답변 늦어짐
3. 임베딩 -> 벡터 데이터베이스에 저장 
4. 질문이 있으면, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [ ]:
## 패키지/라이브러리 설치
# %pip install -qU docx2txt langchain_community langchain-text-splitters langchain-chroma

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import Docx2txtLoader


## 1. 문서 내용 읽기고 분할 #################################################
loader = Docx2txtLoader('law_1.docx')

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

document_list = loader.load_and_split(text_splitter=text_splitter)

## 2. 임베딩 -> 백터 데이터베이스에 저장 ####################################
## 2.1. 환경변수 읽어오기
load_dotenv()

## 2.2. 임베딩 모델 지정  
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

## 2.3 벡터 데이터베이스에 저장
## [방법 1] in memory
database = Chroma.from_documents(
    documents=document_list,
    embedding=embedding,
)

## 3. 질문이 있으면, 벡터 데이터베이스에서 유사도 검색 ######################
# 3.1. 사용자 질문
query = '전세사기피해자 금융지원에 대해 설명해주세요.'

## 3.2. 벡터 데이터베이스에서 유사도 검색
retrieved_docs = database.similarity_search(query=query)

## 4. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 ###################
## 4.1. 프롬프트 작성  
prompt = '''
[identity]
- 당신은 전세사기피해 법률 전문가입니다.
-[context]를 참고하여 사용자의 질문에 답변해주세요.

[context]
{retrieved_docs}

Question : {query}
'''

## 4.2. 프롬프트 변수에 값 설정
formatted_prompt = prompt.format(retrieved_docs=retrieved_docs, query=query)

## 4.3. LLM 모델 생성(ChatOpenAI 인스턴스 생성)
llm = ChatOpenAI(model='gpt-4o')

## 4.4. LLM 모델에 질문과 검색된 문서를 보냄  
ai_message = llm.invoke(formatted_prompt)
ai_message

AIMessage(content='전세사기피해자에 대한 금융지원은 전세사기피해자법에 따라 국가 및 지방자치단체가 제공하는 다양한 형태의 지원을 포함합니다. 주요 내용은 다음과 같습니다:\n\n1. **자금 융자**: 국가 및 지방자치단체는 전세사기피해자 및 관련 요건을 충족한 임차인의 긴급한 주거 안정을 위해 필요한 자금을 융자하거나 필요한 지원을 할 수 있습니다.\n\n2. **주택도시기금을 통한 융자**: 전세사기피해자가 「주택임대차보호법」에 따른 우선변제를 받지 못해 시급한 지원이 필요하다고 인정되는 경우, 주택도시기금에서 필요한 자금을 융자할 수 있습니다.\n\n3. **대출 등록 유예 및 삭제**: 전세사기피해자가 보증금을 모두 변제받지 못한 경우, 금융회사 등은 신용정보법에 따라 해당 전세 관련 대출에 대한 채무 불이행 및 대위변제의 등록을 유예하거나 삭제할 수 있습니다.\n\n4. **주택저당채권대상 확대**: 전세사기피해자 및 관련 임차인에 대해 주택저당채권대상에 준주택으로 사용되는 오피스텔도 포함됩니다.\n\n이러한 금융지원은 전세사기피해자의 주거 안정을 도모하고, 피해를 줄이기 위한 중요한 수단으로 활용됩니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 330, 'prompt_tokens': 4167, 'total_tokens': 4497, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_0787